In [27]:
import pandas as pd
import re

In [28]:
# Map to build FIPS code
state_to_FIPS = {
    'AL':'01',
    'AK':'02',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'FL':'12',
    'GA':'13',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PA':'42',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'WA':'52',
    'WV':'54',
    'WI':'55',
    'WY':'56',
    'AS':'60',
    'GU':'66',
    'MP':'69',
    'PR':'72',
    'VI':'78',
}

In [54]:
def convert_ORI_to_FIPS(ORI_code):
    county_code = ORI_code[2:5]
    state = ORI_code[0:2]
    state_number = state_to_FIPS[state]
    FIPS_Code = state_number + county_code
    return FIPS_Code

In [60]:
# Read the similarity output files
with open('similarity_output_2016-2020.txt') as f:
    dictionary = {}
    rx = re.compile("-----------------------------FBI AGENCY:")
    while True:
        line = f.readline()
        if line:
            m = rx.search(line)
            if m:
                next_line = f.readline()
                parts = next_line.split(",")
                array = parts[1].split(" ")
                target_agency = array[3].split(":")[1]
                target_county = convert_ORI_to_FIPS(target_agency)
                #target_agency = final_part
                ## Find similar matches
                l = f.readline()
                dictionary[target_county] = []
                for i in range(20):
                    try:
                        l = f.readline()
                        similar_agency = l.split(",")[0]
                        if len(similar_agency) > 3:
                            similar_agency = similar_agency[-8:-1]
                            similar_county = convert_ORI_to_FIPS(similar_agency)
                            #print(similar_county)
                            dictionary[target_county].append(similar_county)
                    except:
                        print("Parsing error")
        else:
            break

In [30]:
# Read the combined csv

combined_df = pd.read_csv("combined_csv_2006_2010_new.csv")

/Users/gauribaraskar/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
combined_df['County_code'] = combined_df.ORI_Code.str[2:5]
combined_df['State'] = combined_df.ORI_Code.str[0:2]
combined_df['State_number'] = combined_df['State'].map(state_to_FIPS)
combined_df['FIPS_Code'] = combined_df['State_number'] + combined_df['County_code']
combined_df.drop(['County_code','State','State_number'],axis=1, inplace=True)

In [32]:
combined_df.drop(['ORI_Code','Month','County','Suburban','MSA','State_Code','Division','Group','Agency_Name','State_Name'],axis=1,inplace=True)
#combined_df.groupby([]).mean()

In [33]:
population_df = combined_df.groupby(['FIPS_Code','Year']).agg({'Population_for_Agency':'mean'}).reset_index()

In [34]:
combined_df.drop(['Population_for_Agency'],axis=1,inplace=True)

In [35]:
combined_df = combined_df.groupby(['FIPS_Code','Year','Offense_Code']).sum()

In [36]:
combined_df.columns

Index(['Adult-White', 'Adult-Black', 'Adult-Indian', 'Adult-Asian',
       'Juvenile-White', 'Juvenile-Black', 'Juvenile-Indian', 'Juvenile-Asian',
       'Adult-Hispanic', 'Adult-Non-Hispanic', 'Juvenile-Hispanic',
       'Juvenile-Non-Hispanic', 'Male-Under-10', 'Male-10-12', 'Male-13-14',
       'Male-15', 'Male-16', 'Male-17', 'Male-18', 'Male-19', 'Male-20',
       'Male-21', 'Male-22', 'Male-23', 'Male-24', 'Male-25-29', 'Male-30-34',
       'Male-35-39', 'Male-40-44', 'Male-45-49', 'Male-50-54', 'Male-55-59',
       'Male-60-64', 'Male-Over-64', 'Female-Under-10', 'Female-10-12',
       'Female-13-14', 'Female-15', 'Female-16', 'Female-17', 'Female-18',
       'Female-19', 'Female-20', 'Female-21', 'Female-22', 'Female-23',
       'Female-24', 'Female-25-29', 'Female-30-34', 'Female-35-39',
       'Female-40-44', 'Female-45-49', 'Female-50-54', 'Female-55-59',
       'Female-60-64', 'Female-Over-64'],
      dtype='object')

In [37]:
combined_df['Arrests']= combined_df.iloc[:, 12:].sum(axis=1)

In [38]:
combined_df.head()

Adult-White  Adult-Black  Adult-Indian  \
FIPS_Code Year Offense_Code                                           
01001     6    01A                   0.0          0.0           0.0   
               01B                   0.0          0.0           0.0   
               02                    0.0          0.0           3.0   
               03                    0.0          0.0           4.0   
               04                    0.0          1.0           4.0   

                             Adult-Asian  Juvenile-White  Juvenile-Black  \
FIPS_Code Year Offense_Code                                                
01001     6    01A                   0.0             1.0             5.0   
               01B                   0.0             0.0             0.0   
               02                    1.0             1.0             0.0   
               03                   10.0            23.0            19.0   
               04                    8.0             7.0            14.0   

                             Juvenile-Indian  Juvenile-Asian  Adult-Hispanic  \
FIPS_Code Year Offense_Code                                                    
01001     6    01A                       8.0             3.0             3.0   
               01B                       0.0             0.0             0.0   
               02                        1.0             4.0             1.0   
               03                       31.0            43.0            27.0   
               04                       37.0            46.0            47.0   

                             Adult-Non-Hispanic  ...  Female-25-29  \
FIPS_Code Year Offense_Code                      ...                 
01001     6    01A                          6.0  ...           0.0   
               01B                          1.0  ...           0.0   
               02                           0.0  ...           0.0   
               03                          19.0  ...           0.0   
               04                          60.0  ...           0.0   

                             Female-30-34  Female-35-39  Female-40-44  \
FIPS_Code Year Offense_Code                                             
01001     6    01A                    0.0           6.0           0.0   
               01B                    0.0           0.0           0.0   
               02                     0.0           5.0           0.0   
               03                     1.0          56.0           0.0   
               04                     3.0          43.0           0.0   

                             Female-45-49  Female-50-54  Female-55-59  \
FIPS_Code Year Offense_Code                                             
01001     6    01A                    1.0           0.0           0.0   
               01B                    0.0           0.0           0.0   
               02                     0.0           0.0           0.0   
               03                     0.0           0.0           0.0   
               04                     0.0           0.0           0.0   

                             Female-60-64  Female-Over-64  Arrests  
FIPS_Code Year Offense_Code                                         
01001     6    01A                    0.0             0.0    155.0  
               01B                    0.0             0.0      2.0  
               02                     0.0             0.0     47.0  
               03                     0.0             0.0    528.0  
               04                     0.0             0.0   1304.0  

[5 rows x 57 columns]

In [39]:
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Female-')))]
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Male-')))]
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Adult-')))]

In [40]:
combined_df = combined_df[combined_df.columns.drop(list(combined_df.filter(regex='Juvenile-')))]

In [41]:
combined_df = combined_df.reset_index()

In [42]:
df = pd.merge(combined_df, population_df, on=["FIPS_Code", "Year"], how="left")

In [43]:
df['Arrest_rate'] = df["Arrests"].div(df["Population_for_Agency"].values)

In [44]:
df.drop(['Arrests','Population_for_Agency','Offense_Code'],axis=1,inplace=True)

In [45]:
df = df.groupby(['FIPS_Code','Year'])['Arrest_rate'].apply(list).reset_index(name='Arrest_rate_by_offense_code')

In [46]:
df = df.groupby(['FIPS_Code'])['Arrest_rate_by_offense_code'].apply(list).reset_index(name='Arrest_rate_by_offense_code_by_year')

In [47]:
df.head()

,FIPS_Code,Arrest_rate_by_offense_code_by_year
0,01001,"[[0.0027748066984642885, 3.580395739953921e-05..."
1,01002,"[[0.00047719264660731964, 4.655538015681167e-0..."
2,01003,"[[0.0008069722401549387, 0.0026630083925112975..."
3,01004,"[[0.00017914160891872898, 4.4785402229682245e-..."
4,01005,"[[0.0003095473595368398, 0.0006190947190736796..."


In [64]:
def get_all_counties():
    all_counties_set = set()
    for key, value in dictionary.items():
        all_counties_set.add(key)
        for v in value:
            all_counties_set.add(v)
    return list(all_counties_set) 

In [65]:
all_counties_list = get_all_counties()

In [72]:
all_counties_list

['02001',
 '48101',
 '01043',
 '05072',
 '04010',
 '04002',
 '06001',
 '01004',
 '06037',
 '01057',
 '01025',
 '01019',
 '04003',
 '48SPD',
 '04013',
 '17CPD',
 '36030',
 '01038',
 '01052',
 '01005',
 '01055',
 '01061',
 '04011',
 '01001',
 '05060',
 '01059',
 '04007',
 '01047',
 '01011',
 '06019',
 '01003',
 '06004',
 '02AST',
 '06007',
 '01041',
 '01002',
 '01044',
 '32002',
 '48HPD',
 '42PEP',
 '01063']

In [67]:
# Filter df to only relevant counties
df = df[df['FIPS_Code'].isin(all_counties_list)]

In [78]:
### Read unemployment file
unemployment = pd.read_csv("Unemployment.csv")

In [83]:
unemployment

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"163,140,305","157,154,185","5,986,120",3.7,"160,611,064","147,677,360","12,933,704",8.1,"65,712",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,...,"2,237,278","2,169,395","67,883",3.0,"2,230,118","2,099,062","131,056",5.9,"51,771",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,"19,805","17,607","2,198",11.1,...,"13,172","11,910","1,262",9.6,NaN,NaN,NaN,NaN,NaN,NaN
3271,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,"2,441","2,120",321,13.2,...,"2,616","2,433",183,7.0,NaN,NaN,NaN,NaN,NaN,NaN
3272,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,"7,636","6,648",988,12.9,...,"7,492","6,307","1,185",15.8,NaN,NaN,NaN,NaN,NaN,NaN
3273,72151,PR,"Yabucoa Municipio, PR",1.0,1.0,1.0,"10,613","9,005","1,608",15.2,...,"8,840","7,685","1,155",13.1,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
unemployment = unemployment[unemployment['FIPS_Code'].isin(all_counties_list)]

In [88]:
unemployment['FIPS_Code'] = unemployment['FIPS_Code'].astype(str)
unemployment[unemployment['FIPS_Code'].isin([x.lstrip('0') for x in all_counties_list])]

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
7,1011,AL,"Bullock County, AL",6.0,6.0,0.0,"3,997","3,656",341,8.5,...,"4,785","4,608",177,3.7,"4,818","4,553",265,5.5,"31,906",61.6
11,1019,AL,"Cherokee County, AL",6.0,6.0,0.0,"11,378","10,862",516,4.5,...,"11,726","11,386",340,2.9,"11,513","10,983",530,4.6,"45,982",88.8
14,1025,AL,"Clarke County, AL",7.0,11.0,0.0,"11,677","10,814",863,7.4,...,"7,588","7,151",437,5.8,"7,769","7,069",700,9.0,"43,822",84.6
22,1041,AL,"Crenshaw County, AL",8.0,6.0,0.0,"6,155","5,800",355,5.8,...,"6,436","6,253",183,2.8,"6,253","5,896",357,5.7,"43,309",83.7
23,1043,AL,"Cullman County, AL",4.0,3.0,0.0,"38,898","37,274","1,624",4.2,...,"38,541","37,533","1,008",2.6,"38,310","36,660","1,650",4.3,"50,897",98.3
25,1047,AL,"Dallas County, AL",4.0,5.0,0.0,"17,948","16,487","1,461",8.1,...,"14,655","13,850",805,5.5,"14,619","13,049","1,570",10.7,"34,034",65.7
29,1055,AL,"Etowah County, AL",3.0,2.0,1.0,"48,496","45,652","2,844",5.9,...,"42,462","41,026","1,436",3.4,"40,724","37,598","3,126",7.7,"43,047",83.1


In [91]:
set([x.lstrip('0') for x in all_counties_list]) - set(unemployment['FIPS_Code'])

{'1002',
 '1004',
 '1038',
 '1044',
 '1052',
 '17CPD',
 '2001',
 '2AST',
 '32002',
 '36030',
 '4002',
 '4010',
 '42PEP',
 '48HPD',
 '48SPD',
 '5060',
 '5072',
 '6004'}

In [99]:
unemployment['FIPS_Code'].unique()[0:10]

array(['0', '1000', '1001', '1003', '1005', '1007', '1009', '1011',
       '1013', '1015'], dtype=object)